In [1]:
import numpy as np
import cv2
import time
from scipy import interpolate, signal, fftpack, optimize
from sklearn.decomposition import PCA
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
from skimage.color import rgb2gray, gray2rgb
from scipy import interpolate, signal
from scipy.signal import butter, lfilter,hann,periodogram
from scipy import stats
import itertools
import math
from skimage.io import imread, imshow
import matplotlib.pyplot as plt

In [ ]:
molarExtinctionCoeffOxy = {}
molarExtinctionCoeffDoxy = {}
with open("extinction_coeff.txt") as f:
    for line in f:
       (lmbda, muoxy,mudoxy) = line.split()
       molarExtinctionCoeffOxy[int(lmbda)] = float(muoxy)
       molarExtinctionCoeffDoxy[int(lmbda)] = float(mudoxy)

In [2]:
def computebackScatteringCoefficient(lmbda):
  mieScattering = 2 * pow(10,-5) * pow(lmbda,-1.5)
  rayleighScattering = 2 * pow(10,12) * pow(lmbda,-4)
  return mieScattering + rayleighScattering

In [3]:
# mu_skin computation 
def computeAbsorptionCoefficientSkin(lmbda):
  a = (lmbda - 154) / 66.2
  return 0.244 + 85.3 * np.exp(-1 * a)

In [4]:

# mu_blood computation 
def computeAbsorptionCoefficientBlood(lmbda):
  muOxy = (2.303 * molarExtinctionCoeffOxy[lmbda] * 150)/64500
  muDoxy = (2.303 * molarExtinctionCoeffDoxy[lmbda] * 150)/64500
  return 0.75 * muOxy + 0.25 * muDoxy 

In [5]:
# mu_dermis computation
def computeAbsorptionCoefficientDermis(lmbda,fblood):
  mublood = computeAbsorptionCoefficientBlood(lmbda)
  muskin = computeAbsorptionCoefficientSkin(lmbda)
  # print(mublood,muskin,lmbda,fblood)
  mudermis = fblood * mublood + (1-fblood) * muskin
  # print(mudermis)
  return mudermis

In [6]:
def computeK(lmbda,fblood):
  k = computeAbsorptionCoefficientDermis(lmbda,fblood)
  s = computebackScatteringCoefficient(lmbda)
  return np.sqrt(k*(k + 2*s))

In [7]:

def computeBeta(lmbda,fblood):
  k = computeAbsorptionCoefficientDermis(lmbda,fblood)
  s = computebackScatteringCoefficient(lmbda)
  return np.sqrt(k/( k + 2*s ))

In [8]:


def computeDk_Dfblood(lmbda):
  mublood = computeAbsorptionCoefficientBlood(lmbda)
  muskin = computeAbsorptionCoefficientSkin(lmbda)
  return mublood - muskin
  

In [9]:


def computeDbeta2_Dk(lmbda,fblood):
  k = computeAbsorptionCoefficientDermis(lmbda,fblood)
  s = computebackScatteringCoefficient(lmbda)
  return (2 * s) / pow(k + 2 * s, 2)


In [10]:
def computeDK_Dk(lmbda,fblood):
  k = computeAbsorptionCoefficientDermis(lmbda,fblood)
  s = computebackScatteringCoefficient(lmbda)
  return ( k + s) / np.sqrt(k*(k + 2 *s ))

In [11]:
def computeDR_DK(lmbda,fblood,d):
  b = computeBeta(lmbda,fblood)
  K = computeK(lmbda,fblood)
  nr = (1 - b * b) * 8 * b* d
  dr = pow((pow((1 + b),2) * np.exp(K*d) - pow((1 - b),2) * np.exp(-1*K*d)),2 )
  return nr/dr

In [12]:

def computeDR_Dbeta2(lmbda,fblood,d):
  b = computeBeta(lmbda,fblood)
  K = computeK(lmbda,fblood)
  dr = (pow((1 + b),2) * np.exp(K*d)) - (pow((1 - b),2) * np.exp(-1*K*d))
  trm2 = 1/dr 
  nr = (pow(b,2) -1) * ((np.exp(K*d)*(1 + 1/b)) - ((1 - 1/b)*np.exp(-1*K*d)))
  trm1 = nr / pow(dr,2)
  return  (trm1 - trm2) * (np.exp(K*d) - np.exp(-1*K*d))  

In [13]:
def computeRdermis(lmbda,fblood,d):
  DR_Dbeta2 = computeDR_Dbeta2(lmbda,fblood,d)
  Dbeta2_Dk = computeDbeta2_Dk(lmbda,fblood)
  DR_DK = computeDR_DK(lmbda,fblood,d)
  DK_Dk = computeDK_Dk(lmbda,fblood)
  Dk_Dfblood = computeDk_Dfblood(lmbda)
  #https://openaccess.thecvf.com/content_ICCV_2017_workshops/papers/w16/Alotaibi_A_Biophysical_3D_ICCV_2017_paper.pdf table1 of this paper deltaFblood mentioned
  deltaFblood = 0.05
  # print("DR_Dbeta2 =",DR_Dbeta2, " Dbeta2_Dk =",Dbeta2_Dk," DR_DK = ",DR_DK, " DK_Dk = ",DK_Dk," Dk_Dfblood = ",Dk_Dfblood)
  Rdermis = ( DR_DK * DK_Dk + DR_Dbeta2 * Dbeta2_Dk ) * Dk_Dfblood * deltaFblood
  return Rdermis 


In [14]:
def computeAbsorptionCoefficientEpidermis(lmbda,fmel):
  mumel = 6.6 * pow(10,11) * pow(lmbda,-3.33)
  muskin = computeAbsorptionCoefficientSkin(lmbda)
  return fmel * mumel + (1-fmel) * muskin

In [15]:
def computeTepidermis(lmbda,fmel):
  muepidermis = computeAbsorptionCoefficientEpidermis(lmbda,fmel)
  return np.exp(-1 * muepidermis)  

In [16]:

def computeTotalReflectance(lmbda,fblood,fmel,d):
  Tepidermis = computeTepidermis(lmbda,fmel)
  Rdermis = computeRdermis(lmbda,fblood,d)
  print("Tepidermis = ",Tepidermis)
  print("Rdermis = ",Rdermis)
  Rtotal = Tepidermis * Tepidermis * Rdermis
  return Rtotal

In [53]:
classifierPath = "./haarcascade_frontalface_default.xml"

In [42]:

# Return a face region
def haar_faces(img, haar_classifier):
    face_params = dict(scaleFactor=1.1, 
                   minNeighbors=9,
                   flags=cv2.CASCADE_SCALE_IMAGE)
    # Detect faces using classifier.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = haar_classifier.detectMultiScale(gray, **face_params)
    
    if len(faces) == 0:
        return []
    # selecting the face with max area
    sizes = [h*w for x, y, w, h in faces]
    if len(sizes) == 0:
        return []
    face_idx = np.argmax(sizes)
    face_max = faces[face_idx]
    return face_max

# Crop the image face out
def crop_face(img, face):
    newimg = img[face[1] + int(0.1 * face[3]):face[1] + int(0.9 *face[3]),
                 face[0] + int(0.25*face[2]) : face[0] + int(0.75*face[2])]       
    return newimg

# Display the image
def display_image(image, name):
    # cv2.namedWindow(name)
    # cv2.imshow(name, image)
    imshow(image)
    plt.show()
    # if (cv2.waitKey(5) == 27):
    #     cv2.destroyWindow(name)
    return

# not yet implemented. (Useful for skin segmentation)
def segment_otsu(image_grayscale, img_BGR):
    threshold_value, threshold_image = cv2.threshold(image_grayscale, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    display_image(threshold_image, "otsu")
    threshold_image_binary = 1- threshold_image/255
    threshold_image_binary = np.repeat(threshold_image_binary[:, :, np.newaxis], 3, axis=2)
    img_face_only = np.multiply(threshold_image_binary, img_BGR)
    return img_face_only

# Return the maximum frequency color within the input image & Draw a histogram of the Color in grayscale picture
def getMaxColor(face_img):
    face_grayscale = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    # plt.hist(face_grayscale.ravel(), 256, [0, 256])
    # plt.show()
    hist,bins = np.histogram(face_grayscale.ravel(),256,[0,256])
    mask = (face_grayscale.ravel() <= (np.argmax(hist) + 1)) * (face_grayscale.ravel() >= (np.argmax(hist) - 1))
    mask_reshaped = mask.reshape((face_grayscale.shape[0], face_grayscale.shape[1], 1))
    face_masked = face_img * mask_reshaped
    axis1sum = np.sum(face_masked, axis = 1)
    axis2sum = np.sum(axis1sum, axis = 0)
    return axis2sum / sum(mask)

# Draw a Color Patch
def drawSkinTone(RGB_value, name):
    image = np.zeros((300, 300, 3), np.uint8)
    image[:] = (np.round(RGB_value[0]), np.round(RGB_value[1]), np.round(RGB_value[2]))
    display_image(image, name)
    return

# Highlight Removal Algorithm.
# Follow the algorithm on https://link.springer.com/content/pdf/10.1007%2F978-3-642-15561-1_7.pdf
def compute_theta(input_img):
    output_img = np.amax(input_img, axis = 2) / (np.sum(input_img, axis = 2) + 10e-7)
    return output_img

def compute_lemda(input_img):
    min_pixel = np.amin(input_img, axis = 2) / (np.sum(input_img, axis = 2) + 10e-7)
    min_pixel = np.expand_dims(min_pixel, axis = 2)
    temp = np.amax((input_img / np.expand_dims(np.sum(input_img, axis = 2) + 10e-7, axis = 2) - min_pixel) / (1 - 3 * min_pixel), axis = 2)
    return temp

In [18]:
#http://www.easyrgb.com/en/math.php
def rgb2lab(sR,sG,sB):
  var_R = ( sR / 255.0 )
  var_G = ( sG / 255.0 )
  var_B = ( sB / 255.0 )

  if ( var_R > 0.04045 ): 
    var_R = pow(( var_R + 0.055 ) / 1.055 , 2.4)   
  else:                   
    var_R = var_R / 12.92
  if ( var_G > 0.04045 ):
     var_G = pow(( ( var_G + 0.055 ) / 1.055 ),2.4)
  else:
    var_G = var_G / 12.92
  if ( var_B > 0.04045 ):
    var_B = pow(( ( var_B + 0.055 ) / 1.055 ),2.4)
  else:
    var_B = var_B / 12.92

  var_R = var_R * 100
  var_G = var_G * 100
  var_B = var_B * 100

  X = var_R * 0.4124 + var_G * 0.3576 + var_B * 0.1805
  Y = var_R * 0.2126 + var_G * 0.7152 + var_B * 0.0722
  Z = var_R * 0.0193 + var_G * 0.1192 + var_B * 0.9505
  ReferenceX = 95.047	
  ReferenceY = 100.000	
  ReferenceZ = 108.883
  var_X = X / ReferenceX
  var_Y = Y / ReferenceY
  var_Z = Z / ReferenceZ

  if ( var_X > 0.008856 ):
    var_X = pow(var_X,( 1/3 ))
  else:
    var_X = ( 7.787 * var_X ) + ( 16 / 116 )
  if ( var_Y > 0.008856 ):
    var_Y = pow(var_Y,( 1/3 ))
  else:
    var_Y = ( 7.787 * var_Y ) + ( 16 / 116 )
  if ( var_Z > 0.008856 ):
    var_Z = pow(var_Z,( 1/3 ))
  else:
    var_Z = ( 7.787 * var_Z ) + ( 16 / 116 )

  L = ( 116 * var_Y ) - 16
  a = 500 * ( var_X - var_Y )
  b = 200 * ( var_Y - var_Z )
  return L,a,b

In [19]:
#http://krvarshney.github.io/pubs/KinyanjuiOCCPSV_fmlh2019.pdf  (Skin tone computation)
def skinToneClassification(L,a,b):
  #tone 0 - light, 1 - moderate, 2 - dark
  tone = 0 
  ITA = np.arctan((L -50)/b) * (180/math.pi)
  if(ITA >= 51):
    tone = 0
  elif(ITA >= 28):
    tone = 1
  else:
    tone = 2
  return tone,ITA

In [20]:
#https://omlc.org/news/jan98/skinoptics.html
#https://omlc.org/news/dec14/Jacques_PMB2013/Jacques_PMB2013.pdf
#table 3
#based on skin tone
def computevolumefractionMelanosomes(tone):
  fmel = 3.3 # 0.87 
  if tone == 0:
    fmel = 3.3 # 0.87  
  elif tone == 1:
    fmel = 13.5 # 1.15  
  else:
    fmel = 30.5  # 1.65 
  return fmel/100

In [21]:
#https://omlc.org/news/dec14/Jacques_PMB2013/Jacques_PMB2013.pdf
def computevolumefractionBlood(tone):
  fblood =  0.34
  if tone == 0:
    fblood = 0.34
  elif tone == 1:
    fblood = 0.41
  else:
    fblood = 0.12
  return fblood/100

In [22]:

def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

In [23]:

def make_filter(order=5, low_freq=0.75, high_freq=5, sample_freq=250.0):
  nyq = 0.5 * sample_freq
  low = low_freq / nyq
  high = high_freq / nyq
  b, a = signal.butter(order, [low, high], btype='bandpass')
  func = lambda x: signal.lfilter(b, a, x)
  func.b = b
  func.a = a
  return func

In [24]:

def prpsd(BVP, FS, LL_PR, UL_PR):
  Nyquist = FS/2
  FResBPM = 0.5 #resolution (bpm) of bins in power spectrum used to determine PR and SNR

  N = (60*2*Nyquist)/FResBPM
  print("periodogram comp start")
  print(BVP.shape)
  window = np.hamming(BVP.shape[0]).ravel()
  print(window.shape)
  F,Pxx = periodogram(x = BVP.ravel(),fs = FS,window = window,nfft = N)
  print("periodogram comp end")
  FMask = (F >= (LL_PR/60))&(F <= (UL_PR/60))
  FRange = F[FMask]
  PRange = Pxx[FMask]
  # MaxInd = argmax(Pxx(FMask),1);
  MaxInd = np.argmax(Pxx[FMask])
  PR_F = FRange[MaxInd]
  PR = PR_F*60
  return PR
  


In [25]:
def calPeriodicity(X,FS):
  # X = X / np.max(X)
  X -= np.mean(X)
  spd = np.abs(np.fft.rfft(X))**2
  
  # spd = spd[1:, :]

  L = X.shape[0]
        
  freqs = float(FS) / L * np.arange(L / 2 + 1)
  freqs_in_minute = 60. * freqs
  # freqs_in_minute = freqs_in_minute[1:-1]
  interest_idx = np.where((freqs_in_minute > 42) & (freqs_in_minute < 180))[0]

  interest_idx_sub = interest_idx[:-1].copy() #advoid the indexing error
  freqs_of_interest = freqs_in_minute[interest_idx_sub]
  spd_of_interest = spd[interest_idx_sub]
  
  # maxPwrSrc = np.max(spd, axis=1)
  # validPwr = maxPwrSrc[interest_idx_sub]
  
  maxPwrIdx = np.argmax(spd_of_interest)
  return freqs_of_interest[maxPwrIdx]

In [46]:
 def CHROM_DEHAAN(VideoFile,classifierPath,Weighted_CHROM,FramesToRead):
#       VideoFile               = Video file path.
#       FS                      = Video framerate (fps).
  cap = cv2.VideoCapture(VideoFile)
  FS = cap.get(cv2.CAP_PROP_FPS)
  FS = float(FS)
  SkinSegmentTF=True
  LPF = 0.7 #low cutoff frequency (Hz) - specified as 40 bpm (~0.667 Hz) in reference
  HPF = 2.5 #high cutoff frequency (Hz) - specified as 240 bpm (~4.0 Hz) in reference
  
  WinSec = 1.6 #1.6 #(was a 32 frame window with 20 fps camera)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  FramesToRead = 200
  print("Total Frames = ",FramesToRead)
  T = np.zeros((FramesToRead,1))   #initialize time vector
  RGB = np.zeros((FramesToRead,3)) #initialize color signal
  FN = 0  # current frame number

  haar_cascade = cv2.CascadeClassifier(classifierPath)

  fmel = 0
  fblood = 0

  #wavelength in nm
  Rlambda = 650
  Glambda = 550
  Blambda = 450
  RtotalRed = 0
  RtotalGreen = 0
  RtotalBlue = 0

  eps = pow(10,-5)
  frames = {}
  while(cap.isOpened()):
    # read frame by frame
    ret, VidFrame = cap.read()
    
    if ret == True:
      
      T[FN] = FN/FS
      capture = cv2.cvtColor(VidFrame, cv2.COLOR_BGR2RGB)
      
      # VidROI = capture
      face_rect = haar_faces(capture, haar_cascade)
      if len(face_rect) == 0:
        continue
      # See if the classification fails
      face_only = crop_face(capture, face_rect)
      VidROI = face_only
      VidROI = np.array(face_only)
      VidROI.astype(float)
      
      dominantSkinTone = getMaxColor(face_only)
      # Define Variance of Color and Position Spaces
      var_R = 0.04
      var_S = 0.25
      theta_img = compute_theta(capture)
      theta_img = theta_img.astype('f')
      lemda_img = compute_lemda(capture)
      lemda_img = lemda_img.astype('f')
      # Apply Joint Bilaterial Filtering. Theta as Input and Lemda as Reference
      filtered = cv2.ximgproc.jointBilateralFilter(np.expand_dims(theta_img, axis = 2), np.expand_dims(lemda_img, axis = 2), -1, var_R, var_S)
      # Follow the research at https://link.springer.com/content/pdf/10.1007%2F978-3-642-15561-1_7.pdf
      # Subtract the Specular from total.

      subtract = np.amax(capture, axis = 2) - filtered * np.sum(capture, axis = 2)
      reduce = subtract / (1 - 3 * filtered)
      diffuse_img = capture - np.expand_dims(reduce, axis = 2)
      reduce = reduce / 256
      # display_image(reduce, "Specular Highlight")
      diffuse_img = diffuse_img.astype('uint8')
      # display_image(diffuse_img, "Diffuse")

      # Get the skin tone after specular highlight removal
      faceDiffuse = crop_face(diffuse_img, face_rect)
      spectral_intensity = crop_face(reduce, face_rect)
      spectral_intensity.astype(float)
      dominantSkinToneAfter = getMaxColor(faceDiffuse)
      spectral_intensity = np.expand_dims(spectral_intensity, axis = 2)
      # spectral_intensity = spectral_intensity/ np.sum(spectral_intensity, dtype = np.float32)
      d = 0.2 # depth of epidermis
      L,a,b = rgb2lab(dominantSkinToneAfter[0],dominantSkinToneAfter[1],dominantSkinToneAfter[2])
      tone,ITA = skinToneClassification(L,a,b)
      tone = 0

      VidROI2 = VidROI
      if(Weighted_CHROM == 1):
        VidROI2 = VidROI / (spectral_intensity + eps)
      # frames[FN] = VidROI
      if(SkinSegmentTF): 
        YCBCR = rgb2ycbcr(VidROI)       # use original value for skin segmentation w/o adding specular noise weight
        Yth = YCBCR[:,:,0]>80
        CBth = (YCBCR[:,:,1]>77)*(YCBCR[:,:,1]<127)
        CRth = (YCBCR[:,:,2]>133)*(YCBCR[:,:,2]<173)
        temp = Yth*CBth*CRth
        temp = np.expand_dims(temp, axis=-1)
        temp = np.tile(np.uint8(temp),[1,1,3])
        ROISkin = VidROI2 * temp
        mask = np.uint8(ROISkin>0)
        RGB[FN,:] = (np.sum(np.sum(ROISkin,axis = 0),axis = 0)).squeeze() / np.sum(np.sum(mask,axis = 0),axis = 0).squeeze()
        
      else:
        RGB[FN,:] = np.sum(np.sum(VidROI2,axis = 1),axis = 0) / (VidROI2.shape[0]*VidROI2.shape[1])
      FN = FN+1
      if(FN==FramesToRead):
        break
    else:
      break
  cap.release()
  
  NyquistF = 1/2*FS
  butter_filter = make_filter(order=3,low_freq=LPF, high_freq=HPF,sample_freq=FS)
  WinL = int(np.ceil(WinSec*FS))
  if(WinL%2 == 1): #force even window size for overlap, add of hanning windowed signals
    WinL=WinL+1
  
  NWin = int(np.floor((FN-WinL/2)/(WinL/2)))


  S = np.zeros((int(FN),1))

  WinS = int(1)  #Window Start Index
  WinM = int(WinS+WinL/2)    #Window Middle Index
  WinE = int(WinS+WinL-1)    #Window End Index
  for i in range(0,NWin):
    
    TWin = T[WinS-1:WinE,:]
   
    RGBBase = np.mean(RGB[WinS-1:WinE,:])
    # RGBNorm = bsxfun(@times,RGB(WinS:WinE,:),1./RGBBase)-1;              ### start from here 
    RGBNorm = (RGB[WinS-1:WinE,:] * (1/RGBBase)) -1 
    # CHROM
    Xs = (3*RGBNorm[:,0]-2*RGBNorm[:,1]).squeeze()   #3Rn-2Gn
    Ys = (1.5*RGBNorm[:,0]+RGBNorm[:,1]-1.5*RGBNorm[:,2]).squeeze()    #1.5Rn+Gn-1.5Bn
    
    Xf = butter_filter(Xs)
    Yf = butter_filter(Ys)
    
    Alpha = np.std(Xf)/np.std(Yf)
    
    SWin = Xf - Alpha*Yf
    
    SWin = signal.hann(WinL)*SWin
    SWin = np.array(SWin).reshape((WinL,1))
    # %overlap, add Hanning windowed signals
    if(i==-1):
      S = SWin
      TX = TWin
    else:
      # print("WinM-1 =",WinM-1," WinE =",WinE, "S.shape = ",S[WinS-1:WinM-1].shape, SWin[0:int(WinL/2)].shape)
      b = SWin[0:int(WinL/2)] + S[WinS-1:WinM-1]   #1st half overlap
      
      S[WinS-1:WinM-1] = b[:]
      S[WinM-1:WinE] = SWin[int(WinL/2):]                  #2nd half
      # TX[WinM-1:WinE] = TWin[int(WinL/2):]
    
    WinS = int(WinM)
    WinM = int(WinS+WinL/2)
    WinE = int(WinS+WinL-1)
    print(WinS,WinM,WinE,SWin.shape[0],S.shape[0])
  
  BVP=S
  T=T[0:BVP.shape[0]]
  
  PR = prpsd(BVP,FS,40,240)
  PR2 = calPeriodicity(BVP,FS)
  return PR, BVP
  # return frames



In [ ]:
haar_cascade = cv2.CascadeClassifier(classifierPath)
capture = cv2.imread(filePath)
face_rect = haar_faces(capture, haar_cascade)
# See if the classification fails
face_only = crop_face(capture, face_rect)

In [28]:
import cv2
import pandas as pd
import numpy as np
import os
from skimage.io import imread, imshow

from matplotlib import pyplot as plt
import matplotlib.patches as patches
from mtcnn import MTCNN
import time

from imutils import face_utils
import numpy as np
import imutils
import dlib
import skimage
import cv2
import matplotlib.pyplot as plt
import heartpy as hp

In [50]:
videoPath = './Project Videos/set2/video_bottom.mp4'
groundTruth = pd.read_csv('./Project Videos/set2/MPDataExport.csv')

classifierPath = "./haarcascade.xml"
haar_cascade = cv2.CascadeClassifier(classifierPath)

In [51]:
pr, BVP = CHROM_DEHAAN(videoPath, classifierPath, 0, 200)

Total Frames =  200
25 49 72 48 200
49 73 96 48 200
73 97 120 48 200
97 121 144 48 200
121 145 168 48 200
145 169 192 48 200
169 193 216 48 200
periodogram comp start
(200, 1)
(200,)
periodogram comp end
